In [1]:
from pandas_ods_reader import read_ods
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
hcdata = read_ods('HC-Data.ods', 'Sheet1')

In [3]:
hcdata.dtypes

Date     object
H1      float64
H2      float64
H3      float64
H4      float64
H5      float64
H6      float64
H7      float64
H8      float64
H9      float64
H10     float64
H11     float64
H12     float64
H13     float64
H14     float64
H15     float64
H16     float64
dtype: object

In [4]:
hcdata.head()

,Date,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10,H11,H12,H13,H14,H15,H16
0,2020-09-21,22.0,22.0,23.0,23.0,23.0,23.0,19.0,15.0,16.0,17.0,20.0,20.0,20.0,20.0,20.0,20.0
1,2020-09-22,20.0,20.0,21.0,21.0,21.0,21.0,17.0,13.0,11.0,13.0,11.0,11.0,14.0,13.0,13.0,17.0
2,2020-09-23,23.0,23.0,22.0,15.0,14.0,14.0,13.0,9.0,14.0,12.0,10.0,11.0,12.0,8.0,10.0,12.0
3,2020-09-24,18.0,18.0,18.0,17.0,17.0,18.0,14.0,16.0,13.0,11.0,14.0,15.0,15.0,13.0,15.0,16.0
4,2020-09-25,25.0,24.0,19.0,18.0,20.0,20.0,17.0,13.0,15.0,12.0,18.0,18.0,19.0,19.0,18.0,19.0


In [5]:
def resample(T, available_freq, desired_freq):
    factor = int(available_freq/desired_freq)

    T1 = [[n] * factor for n in T]
    T2 = [int(n) for sublist in T1 for n in sublist]

    return T2


In [6]:
hcdata.iloc[0, :][1:]

H1     22
H2     22
H3     23
H4     23
H5     23
H6     23
H7     19
H8     15
H9     16
H10    17
H11    20
H12    20
H13    20
H14    20
H15    20
H16    20
Name: 0, dtype: object

In [7]:
import datetime
dt = datetime.date.fromisoformat(hcdata.iloc[1,:][0])

In [8]:
dt.weekday()

1

In [9]:
desired_freq = 3

In [10]:
agent_count = resample(hcdata.iloc[0,:][1:], 30, desired_freq)
Period = [hcdata.iloc[0,:][0]] * len(agent_count)

In [11]:
agent_count = []
period = []
isoday_of_week = []
for i in range(hcdata.shape[0]):
    X = resample(hcdata.iloc[i,:][1:], 30, desired_freq)
    Y = [hcdata.iloc[i,:][0]] * len(X)
    dt = datetime.date.fromisoformat(hcdata.iloc[i,:][0])
    Z = [dt.isoweekday()] * len(X)
    agent_count.append(X)
    period.append(Y)
    isoday_of_week.append(Z)

In [12]:
agent_count = [ac for sublist in agent_count for ac in sublist]

In [13]:
period = [p for sublist in period for p in sublist]
isoday_of_week = [d for sublist in isoday_of_week for d in sublist]

In [14]:
len(agent_count)

4480

In [15]:
len(period)

4480

In [16]:
X = resample(hcdata.iloc[0,:][1:], 30, desired_freq)
len(X)

160

In [17]:
A = pd.DataFrame({'Period': period, 'agent_count': agent_count, 'isoday_of_week': isoday_of_week})

In [18]:
B = pd.read_csv('consolidated-1.csv')

In [19]:
A.dtypes

Period            object
agent_count        int64
isoday_of_week     int64
dtype: object

In [20]:
B.dtypes

Create-time     object
ncalls           int64
Avg-time       float64
Period          object
nagents          int64
office-hour       bool
dtype: object

In [21]:
A.shape

(4480, 3)

In [22]:
A.head()

,Period,agent_count,isoday_of_week
0,2020-09-21,22,1
1,2020-09-21,22,1
2,2020-09-21,22,1
3,2020-09-21,22,1
4,2020-09-21,22,1


In [23]:
C = A[A['isoday_of_week'] < 6]

In [24]:
C.shape

(3200, 3)

In [25]:
C[['Period', 'agent_count']].groupby('Period').count()

,agent_count
Period,
2020-09-21,160
2020-09-22,160
2020-09-23,160
2020-09-24,160
2020-09-25,160
2020-09-28,160
2020-09-29,160
2020-09-30,160
2020-10-01,160


In [26]:
C.to_csv('HC-Data-1.csv', index=False)

**Important**
Create the file 'consolidated-2.csv' manually from 'consolidated-1.csv' and 'HC-Data-1.csv'. Use 'consolidated-2.csv' for the final analysis.